## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
import gmaps.datasets

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,3,East London,ZA,69.42,light rain,-33.0153,27.9116,Tu Casa
1,4,Mount Gambier,AU,58.91,overcast clouds,-37.8333,140.7667,The Commodore
2,7,Sao Felix Do Xingu,BR,74.57,moderate rain,-6.6447,-51.9950,Rio Xingu Camping
3,11,Ushuaia,AR,55.06,light rain,-54.8000,-68.3000,Albatros Hotel
4,13,Punta Arenas,CL,57.31,scattered clouds,-53.1500,-70.9167,Hotel Hain


In [3]:
clean_vacation_df = vacation_df.dropna()
clean_vacation_df.isnull().sum()

City_ID                0
City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [4]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_vacation_df[["Lat", "Lng"]]

In [5]:
# 4a. Add a marker layer for each city to the map.
max_temp = clean_vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [49]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = clean_vacation_df.loc[clean_vacation_df['City'] == 'Virginia Beach']
vacation_end = clean_vacation_df.loc[clean_vacation_df['City'] == 'Virginia Beach']
vacation_stop1 = clean_vacation_df.loc[clean_vacation_df['City'] == 'Havelock']
vacation_stop2 = clean_vacation_df.loc[clean_vacation_df['City'] == 'Charlottesville']
vacation_stop3 = clean_vacation_df.loc[clean_vacation_df['City'] == 'Mayo']
vacation_start

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
114,397,Virginia Beach,US,57.42,clear sky,36.8529,-75.978,Hilton Virginia Beach Oceanfront


In [50]:
vacation_stop1

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
92,334,Havelock,US,64.36,clear sky,34.8791,-76.9013,Sherwood Motel


In [51]:
vacation_stop2

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
179,625,Charlottesville,US,56.21,clear sky,38.0293,-78.4767,Hampton Inn & Suites Charlottesville-At The Un...


In [52]:
vacation_stop3

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
112,394,Mayo,US,55.56,clear sky,38.8876,-76.5119,Resorts Resource Group International


In [35]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start[['Lat','Lng']].to_numpy()[0]
end = vacation_end[['Lat','Lng']].to_numpy()[0]
stop1 = vacation_stop1[['Lat','Lng']].to_numpy()[0]
stop2 = vacation_stop2[['Lat','Lng']].to_numpy()[0]
stop3 = vacation_stop3[['Lat','Lng']].to_numpy()[0]
start, end, stop1, stop2, stop3

(array([ 36.8529, -75.978 ]),
 array([ 36.8529, -75.978 ]),
 array([ 34.8791, -76.9013]),
 array([ 38.0293, -78.4767]),
 array([ 38.8876, -76.5119]))

In [36]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
vacay_trip = gmaps.directions_layer(
        start, end, waypoints=[stop1,stop2,stop3],
        travel_mode='DRIVING')
fig.add_layer(vacay_trip)
fig

Figure(layout=FigureLayout(height='420px'))

In [38]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start,vacation_stop1,vacation_stop2,vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,397,Virginia Beach,US,57.42,clear sky,36.8529,-75.9780,Hilton Virginia Beach Oceanfront
1,334,Havelock,US,64.36,clear sky,34.8791,-76.9013,Sherwood Motel
2,625,Charlottesville,US,56.21,clear sky,38.0293,-78.4767,Hampton Inn & Suites Charlottesville-At The Un...
3,394,Mayo,US,55.56,clear sky,38.8876,-76.5119,Resorts Resource Group International


In [53]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [54]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))